# The simple, joyful experience of an Agent Loop



### Start with some imports

In [ ]:
import os
from agents import Agent, Runner, function_tool
from rich.console import Console

: 

### Check that the OpenAI API key is set from the environment

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key or not openai_api_key.startswith("sk-"):
    print("No OPENAI_API_KEY environment variable set - please see the README for details")
else:
    print("OPENAI_API_KEY looks good")

In [ ]:
agent = Agent("Joke teller", model="gpt-5.1-mini")
result = await Runner.run(agent, "Tell me a joke for some data scientists")
print(result.final_output)

In [ ]:
todos = []
completed = []

In [ ]:
def get_todo_report(print: bool=False) -> str:
    """Get a report of all todos."""
    result = ""
    for index, (todo, complete) in enumerate(zip(todos, completed)):
        check = "X" if complete else " "
        start = "[strike][green]" if complete else ""
        end = "[/strike][/green]" if complete else ""
        result += f"Todo #{index + 1}: [{check}] {start}{todo}{end}\n"
    if print:
        Console().print(result)
    return result

In [ ]:
get_todo_report()

In [ ]:
def create_todos(descriptions: list[str]) -> str:
    """Add new todos from a list of descriptions and return the full list"""
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report(print=True)

def mark_complete(index: int) -> str:
    """Mark complete the todo at the given position (starting from 1) and return the full list"""
    if 1 <= index <= len(todos):
        completed[index - 1] = True
    else:
        return "No todo at this index."
    return get_todo_report(print=True)

In [ ]:
todos, completed = [], []

create_todos(["Buy groceries", "Finish lab1", "Go for a walk"])

In [ ]:
mark_complete(1)

In [ ]:
@function_tool
def create_todos(descriptions: list[str]) -> str:
    """Add new todos from a list of descriptions and return the full list"""
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report(print=True)

@function_tool
def mark_complete(index: int, completion_notes: str) -> str:
    """Mark complete the todo at the given position (starting from 1) and return the full list
    
    Args:
        index: The 1-based index of the todo to mark as complete
        completion_notes: Notes about how you completed the todo in rich console markup
    """
    if 1 <= index <= len(todos):
        completed[index - 1] = True
    else:
        return "No todo at this index."
    Console().print(completion_notes)
    return get_todo_report(print=True)

In [ ]:
mark_complete.params_json_schema

In [ ]:
instructions = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
Provide your solution in Rich console markup (e.g. [bold red]Error[/bold red]) to indicate colors and styles.
"""
tools = [create_todos, mark_complete]
agent = Agent("Puzzle Agent", model="gpt-4.1-mini", instructions=instructions, tools=tools)

In [ ]:
task = "A train leaves Boston at 2:00 pm traveling 60 mph. Another train leaves New York at 3:00 pm traveling 80 mph toward Boston. When do they meet?"
todos, completed = [], []
response = await Runner.run(agent, task)
Console().print("\n\n" + response.final_output)